In [13]:
import numpy as np
import pandas as pd
from alabtools.analysis import HssFile
from matplotlib import pyplot as plt
%matplotlib inline

#### Solution for the assignment

At individual structure level, in structure $m$, the distance of chromatin region $i$ to the nuclear center $d_{i,m}$ is 
$$
d_{i,m}=\frac{\sum_{c=0}^{K}{d_{c,i,m}}}{K}
$$
where $d_{i,c,m}$ is the distance of copy $c$ of region $i$ to the nuclear center, so d_{i,m} is the average of multiple copies of region $i$ to the nuclear center in structure $m$. For example, if genome is diploid
$$
d_{i,m}=\frac{d_{0,i,m}+d_{1,i,m}}{2}
$$
Because different cells have different nucleus shape, in order to make the radial position of a chromatin regions comparable within and across different cell. We use the normalized version of $d_{i,m}$:
$$
r_{i,m} = \frac{d_{i,m}}{R^{nuc}}
$$
where $R^{nuc}$ is the nucleus radius which is 5 μm for a sphere nucleus. $r_{i,m}$
= 0 means the region i is at the nuclear center while $r_{i,m}=1$ means it is located at the nuclear surface. 

### Mean radial position (RAD)

How can we measure the average radial position of a chromatin region $i$ across all the structures in the population? We define the mean radial position RAD
$$
RAD_{i} = \frac{\sum_{m=1}^{N}{r_{i,m}}}{N}
$$
Where $N$ is the total number of structures in the population

In [14]:
# Read in HSS file
hss = HssFile('igm-model_mcrb_2.5MB.hss', 'r')
# First, we need to know which beads that represent the diffrent copies belong to the same bin in genome
copy_index=hss.index.copy_index
print(copy_index)
print(len(copy_index))

{0: [0, 1100], 1: [1, 1101], 2: [2, 1102], 3: [3, 1103], 4: [4, 1104], 5: [5, 1105], 6: [6, 1106], 7: [7, 1107], 8: [8, 1108], 9: [9, 1109], 10: [10, 1110], 11: [11, 1111], 12: [12, 1112], 13: [13, 1113], 14: [14, 1114], 15: [15, 1115], 16: [16, 1116], 17: [17, 1117], 18: [18, 1118], 19: [19, 1119], 20: [20, 1120], 21: [21, 1121], 22: [22, 1122], 23: [23, 1123], 24: [24, 1124], 25: [25, 1125], 26: [26, 1126], 27: [27, 1127], 28: [28, 1128], 29: [29, 1129], 30: [30, 1130], 31: [31, 1131], 32: [32, 1132], 33: [33, 1133], 34: [34, 1134], 35: [35, 1135], 36: [36, 1136], 37: [37, 1137], 38: [38, 1138], 39: [39, 1139], 40: [40, 1140], 41: [41, 1141], 42: [42, 1142], 43: [43, 1143], 44: [44, 1144], 45: [45, 1145], 46: [46, 1146], 47: [47, 1147], 48: [48, 1148], 49: [49, 1149], 50: [50, 1150], 51: [51, 1151], 52: [52, 1152], 53: [53, 1153], 54: [54, 1154], 55: [55, 1155], 56: [56, 1156], 57: [57, 1157], 58: [58, 1158], 59: [59, 1159], 60: [60, 1160], 61: [61, 1161], 62: [62, 1162], 63: [63, 11

In [15]:
# get chromosome names, start and end positions of the bins
index = hss.index
index_chroms = index.chromstr
index_starts = index.start
index_ends = index.end

In [37]:
crds=hss.coordinates
# iterate keys over copy_index, and get the coordinates of the beads that belong to the same bin, 
# then get the average radials of the beads that belong to the same bin, write chr start end and average 
# radial to a file
for bin in copy_index:
    #get chr, start, end, of the bin
    bead_0=copy_index[bin][0]
    chr=index_chroms[bead_0]
    start=index_starts[bead_0]
    end=index_ends[bead_0]
    #get the radials of the beads that belong to the same bin
    radials = hss.getBeadRadialPositions(copy_index[bin], nucleusRadius=(3050, 2350, 2350))
    #write chr start end and average radial to a file
    with open('radial.txt', 'a') as f:
        f.write(chr+'\t'+str(start)+'\t'+str(end)+'\t'+str(np.mean(radials))+'\n')
    f.close()

#### Variability of structual feature
Now we know each genomic loci (bin) has an average radial position (RAD) in nucleus. Apart from the "Mean" of feature, Is there any other type of information we can get from the structure?

We may use standard deviation to measure the disperson of radials for a particular bin $i$ in individual structures
$$
\sigma_i = \sqrt{\frac{1}{N}\frac{1}{K}\sum_{m=1}^{N}\sum_{c=0}^{K}(r_{c,i,m} - RAD_i)^2}
$$

In [17]:
#compute the std of the radials of the beads that belong to the same bin
for bin in copy_index:
    #get chr, start, end, of the bin
    bead_0=copy_index[bin][0]
    chr=index_chroms[bead_0]
    start=index_starts[bead_0]
    end=index_ends[bead_0]
    #get the radials of the beads that belong to the same bin
    radials = hss.getBeadRadialPositions(copy_index[bin], nucleusRadius=(3050, 2350, 2350))
    #write chr start end and average radial to a file
    with open('radial_std.txt', 'a') as f:
        f.write(chr+'\t'+str(start)+'\t'+str(end)+'\t'+str(np.std(radials))+'\n')
    f.close()

Is it possible to make the result more intuitive? We can do it by normalizing the $\sigma_i$ by the mean of $\sigma_i$ across genome ($\sigma_g$). Furthermore, we may take a logarithmic scale to the result, thus a positive value would then indicate that $\sigma_i$ for particular region $i$ exceeds the average variability across all bins in the genome, vice versa.
$$
\sigma_g=\frac{\sum_{i=1}^{M}\sigma_i}{M}
$$
Where $M$ is the total number of regions (bins) across genome, which depends on bin resolution and genome size.
$$
\delta_i=\log_2{\frac{\sigma_i}{\sigma_g}}
$$

In [18]:
#first, we need compute the mean RAD across the genome
#read the radial_std.txt, which contains the std of radial of each bin
radial_std=pd.read_csv('radial_std.txt', sep='\t', header=None)
#compute the mean RAD across the genome, which is the mean of the fourth column of the radial_std.txt
mean_radial_std=np.mean(radial_std[3])

#compute delta RAD of the bin
for bin in copy_index:
    #get chr, start, end, of the bin
    bead_0=copy_index[bin][0]
    chr=index_chroms[bead_0]
    start=index_starts[bead_0]
    end=index_ends[bead_0]
    #get the radials of the beads that belong to the same bin
    radials = hss.getBeadRadialPositions(copy_index[bin], nucleusRadius=(3050, 2350, 2350))
    #write chr start end and average radial to a file
    with open('radial_delta.txt', 'a') as f:
        f.write(chr+'\t'+str(start)+'\t'+str(end)+'\t'+str(np.log2(np.std(radials)/mean_radial_std))+'\n')
    f.close()

In [ ]:
data=pd.read_csv('radial.txt', sep='\t', header=None)
#plot the delta RAD for chr2
chr2=data[data[0]=='chr2']
chr2_coord=chr2[1]
chr2_radial=chr2[3]
plt.plot(chr2_coord,chr2_radial)
plt.show()


In [ ]:
#Refine this plot
#plot the delta RAD for chr2
chr2=data[data[0]=='chr2']
chr2_coord=chr2[1]/1000000
chr2_radial=chr2[3]
plt.plot(chr2_coord,chr2_radial)
plt.xlabel('chr2 coordinate (Mb)')
plt.ylabel('RAD')
plt.ylim(0.5,0.9)
plt.show()

In [ ]:
#What about chr4?
#plot the delta RAD for chr2
chr4=data[data[0]=='chr4']
chr4_coord=chr4[1]/1000000
chr4_radial=chr4[3]
plt.plot(chr4_coord,chr4_radial)
plt.xlabel('chr4 coordinate (Mb)')
plt.ylabel('RAD')
plt.ylim(0.5,0.9)
plt.show()

In [ ]:
#We can plot chr2 and chr4 together
plt.plot(chr2_coord,chr2_radial,label='chr2')
plt.plot(chr4_coord,chr4_radial,label='chr4')
plt.xlabel('chr coordinate (Mb)')
plt.ylabel('RAD')
plt.legend()
plt.ylim(0.5,0.9)

In [45]:
import os
os.getcwd()

'/u/home/w/wyeast/IGM_tutorial/.ipynb_checkpoints'

### Practice
Can you plot the delta RAD for chr2 and chr4?

### Interior localization frequency (ILF)

For a given chromatin region $i$, the interior localization frequency (ILF) is calculated as:
$$
ILF_i=\frac{n_{r<0.5}}{N}
$$
where $ILF_i$ is the number of structures where either copy of the region $i$ has a radial position lower than 0.5, and $N$ is the total number of structures in our population. 0.5 is a empirical threshold of $RAD$, can be adjusted based on how you define a regions is within "interior" nucleus.

In [35]:

#compute ILF of the bin
for bin in copy_index:
    #get chr, start, end, of the bin
    bead_0=copy_index[bin][0]
    chr=index_chroms[bead_0]
    start=index_starts[bead_0]
    end=index_ends[bead_0]
    #get the radials of the beads that belong to the same bin
    radials = hss.getBeadRadialPositions(copy_index[bin], nucleusRadius=(3050, 2350, 2350))
    #get the boolean array showing if bead radial < 0.5
    interior_boolean=radials<0.5
    #we need to compute the ILF of the bin, and at least one bead belong to this bin has radial < 0.5
    new_array = np.any(interior_boolean, axis=0)
    ILF=np.sum(new_array)/new_array.size
    with open('ilf.txt', 'a') as f:
        f.write(chr+'\t'+str(start)+'\t'+str(end)+'\t'+str(ILF)+'\n')

Let's break it down and see what happens in each step

In [33]:

radials=[[0.1,0.6,0.1,0.6],
 [0.1,0.1,0.1,0.6]]


0.75

### Question
Usually $ILF$ should be correlated to $RAD$. A region with small $RAD$ usually has large $ILF$. So why we need to come up with the concept of $ILF$? Can you come up with a senario where those two concept represent different aspects of a chromatin region?

### Exersice
Plot the $ILF$ and $RAD$ of chr2 at the same plot. Set the yscale as (0,0.9). What information can you get from this plot?